In [1]:
from TradingSystems.Strategy1 import Strategy1
from Broker.Broker import Broker
from keras.models import load_model
ts = Strategy1([[]],[["AAPL.csv","csv"]],"TestSystem","TradingSystem",3,Broker("Test","Test"),"D")

None


In [15]:
import numpy as np
import pandas as pd
from keras.models import load_model
from TradingSystems.featureCreatenClass import featuresGen
tmpData = pd.read_csv("Backtest/Backtest_data/btc_test.csv")
model = load_model('TradingSystems/ml_system/1_lstm_V2_reg.h5')

In [11]:
len(tmpData)

244453

In [12]:
tmpData = pd.read_csv("Backtest/Backtest_data/AAPL.csv")


In [13]:
len(tmpData)

8009

In [10]:
tmpData = tmpData.iloc[int(len(tmpData)/2):]

In [ ]:
tmpData.index = pd.to_datetime(tmpData["timestamp"])
tmpData_features = featuresGen(tmpData)
tmpData_features = tmpData_features.drop(["timestamp","zscores","Target","Label","Asset_ID","index","Open","High","Low","Close","Volume"], axis=1).reset_index(drop=True)
tmpData_features.index = tmpData_features.index.astype("int")
#TODO drop features - Low,Close,Open,High
#TODO sequenzierung
n_future = 1 #Naechste 15min prädizieren
n_past = 16
train_x = []
for i in range(n_past,len(tmpData_features)-n_future+1):
    train_x.append(tmpData_features.iloc[i-n_past:i, 0:tmpData_features.shape[1]])
train_x = np.array(train_x)
trainPredict = model.predict(train_x)
diff_len = len(tmpData) - len(trainPredict)
tmpData = tmpData.iloc[diff_len:,]
tmpData["Target"] = trainPredict
# Tradingsystem

openTrades =  False
counter = 0
#np.zero
position_list = np.zeros([len(tmpData),])
target_stopLoss = [0,0]
tmpData = tmpData.reset_index(drop=True)
tmpData.index = tmpData.index.astype("int")  
for index,row in tmpData.iterrows():
    if (row["Minute"] % 15) == 0:
        if row["Target"] > 0:
            counter = 14
            openTrades = True
            target_stopLoss[0] = row["Target"] * row["Close"] + row["Close"]
            target_stopLoss[1] = row["Close"] * 0.05 + row["Close"]
            position_list[index] = 1
        elif counter > 0:
            print("Error - Überschreitung")
            
    elif counter > 0 and openTrades == True:
        if target_stopLoss[1] >= row["Low"] or target_stopLoss[0] <= row["High"]:
            position_list[index] = -1
            counter = 0
            openTrades = False
        else:
            counter -= 1
            

In [4]:
tmpData["Position"] = position_list

In [7]:
tmpData[["timestamp","Open","High","Low","Close","Volume","Position"]]

,timestamp,Open,High,Low,Close,Volume,Position
0,2020-10-16 10:54:00,11306.801429,11320.000000,11297.33,11309.977143,48.283490,0.0
1,2020-10-16 10:55:00,11310.964574,11325.086588,11299.17,11306.235000,38.785834,0.0
2,2020-10-16 10:56:00,11306.365714,11320.000000,11301.31,11309.995714,24.516476,0.0
3,2020-10-16 10:57:00,11311.217143,11323.000000,11303.00,11310.678571,39.842457,0.0
4,2020-10-16 10:58:00,11311.004811,11322.993680,11303.01,11309.609097,31.227807,0.0
...,...,...,...,...,...,...,...
488900,2021-09-20 23:26:00,42707.930000,42836.330000,42660.05,42778.885000,226.506712,0.0
488901,2021-09-20 23:27:00,42766.432500,42811.000000,42651.40,42779.710000,203.675225,0.0
488902,2021-09-20 23:28:00,42775.731250,42852.000000,42672.80,42700.446250,200.178351,0.0
488903,2021-09-20 23:29:00,42694.572857,42719.980000,42640.00,42659.582416,120.113063,0.0


In [16]:
tmpData.iloc[:1000]

,timestamp,index,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,...,Log_Low,Log_Close,zscores,Minute,Day,Month,Year,DayOfWeek,minute_seasonal,correlation_log_perf
0,2020-10-16 08:38:00,17406273,1,519.0,11318.958333,11325.700000,11312.14,11319.455000,36.049136,11318.863131,...,-1.281727e-04,-0.081092,-0.081092,38,16,10,2020,4,0.000245,0.065135
1,2020-10-16 08:39:00,17406287,1,496.0,11319.958333,11325.700000,11315.98,11319.441667,47.625093,11319.822308,...,3.394007e-04,-0.001440,-0.001440,39,16,10,2020,4,0.003156,-0.014025
2,2020-10-16 08:40:00,17406300,1,560.0,11321.225329,11332.018497,11314.15,11321.046928,50.134674,11321.011512,...,-1.617313e-04,0.115608,0.115608,40,16,10,2020,4,0.004399,0.115572
3,2020-10-16 08:41:00,17406314,1,484.0,11321.504071,11332.018497,11314.29,11322.992472,86.899679,11321.805817,...,1.237381e-05,0.140194,0.140194,41,16,10,2020,4,-0.000353,0.005892
4,2020-10-16 08:42:00,17406328,1,445.0,11322.992472,11332.017306,11316.40,11322.865329,30.430689,11322.959498,...,1.864724e-04,-0.009668,-0.009668,42,16,10,2020,4,0.001699,0.091320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2020-10-17 01:13:00,17419837,1,522.0,11305.948750,11317.000000,11299.30,11308.572500,52.493804,11307.903765,...,7.684844e-04,0.322626,0.322626,13,17,10,2020,5,0.002701,0.001527
996,2020-10-17 01:14:00,17419851,1,414.0,11307.738571,11318.000000,11301.09,11307.724286,24.573549,11308.560633,...,1.584043e-04,-0.061880,-0.061880,14,17,10,2020,5,0.002588,-0.123097
997,2020-10-17 01:15:00,17419864,1,379.0,11308.865000,11319.963436,11301.10,11312.875000,19.771139,11311.113762,...,8.848700e-07,0.372324,0.372324,15,17,10,2020,5,0.008953,-0.053211
998,2020-10-17 01:16:00,17419877,1,374.0,11313.705714,11321.000000,11305.30,11314.622857,12.293382,11314.035202,...,3.715762e-04,0.125993,0.125993,16,17,10,2020,5,-0.000237,0.152156


In [2]:
ts.createSignal()
ts.getSignalDf()

            Date        Open        High         Low       Close       Volume  \
0     1990-01-02    0.252479    0.268595    0.250689    0.266804  228368930.0   
1     1990-01-03    0.272176    0.272176    0.268595    0.268595  259279780.0   
2     1990-01-04    0.273967    0.277548    0.266804    0.269490  276131360.0   
3     1990-01-05    0.270386    0.273967    0.265014    0.270386  153716560.0   
4     1990-01-08    0.268595    0.272176    0.265014    0.272176  126617220.0   
...          ...         ...         ...         ...         ...          ...   
8004  2021-10-07  143.060000  144.215000  142.720000  143.290000   61732656.0   
8005  2021-10-08  144.030000  144.178100  142.560000  142.900000   58773156.0   
8006  2021-10-11  142.270000  144.810000  141.810000  142.810000   64452220.0   
8007  2021-10-12  143.230000  143.250000  141.040100  141.510000   73035860.0   
8008  2021-10-13  141.235000  141.400000  139.200000  140.910000   78762720.0   

          Turnover  Unadjus

,Date,Open,High,Low,Close,Volume,Turnover,Unadjusted Close,Dividend,20_SMA,50_SMA,Signal,Position,Short_Signal,Long_Signal
0,1990-01-02,0.252479,0.268595,0.250689,0.266804,228368930.0,6.092982e+07,37.250,0.0,0.266804,0.266804,0,0.0,NaN,NaN
1,1990-01-03,0.272176,0.272176,0.268595,0.268595,259279780.0,6.964125e+07,37.500,0.0,0.267700,0.267700,0,0.0,NaN,NaN
2,1990-01-04,0.273967,0.277548,0.266804,0.269490,276131360.0,7.441473e+07,37.625,0.0,0.268297,0.268297,0,0.0,NaN,NaN
3,1990-01-05,0.270386,0.273967,0.265014,0.270386,153716560.0,4.156275e+07,37.750,0.0,0.268819,0.268819,0,0.0,NaN,NaN
4,1990-01-08,0.268595,0.272176,0.265014,0.272176,126617220.0,3.446220e+07,38.000,0.0,0.269490,0.269490,0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8004,2021-10-07,143.060000,144.215000,142.720000,143.290000,61732656.0,8.869865e+09,143.290,0.0,144.814500,147.454318,0,0.0,NaN,NaN
8005,2021-10-08,144.030000,144.178100,142.560000,142.900000,58773156.0,8.415117e+09,142.900,0.0,144.511000,147.403876,0,0.0,NaN,NaN
8006,2021-10-11,142.270000,144.810000,141.810000,142.810000,64452220.0,9.263807e+09,142.810,0.0,144.174000,147.347240,0,0.0,NaN,NaN
8007,2021-10-12,143.230000,143.250000,141.040100,141.510000,73035860.0,1.036863e+10,141.510,0.0,143.843500,147.271394,0,0.0,NaN,NaN


In [1]:
import schedule
import datetime
import time

def job():
    print("I'm working...")


now = datetime.datetime.now()
print(((((5-(now.minute % 5))-1) * 60) + (60-now.second))/60)
#time.sleep((((5-(now.minute % 5))-1) * 60) + (60-now.second))
print("test")
#schedule.every(5).minutes.do(job)

while False:
    schedule.run_pending()
    time.sleep(1)

3.9833333333333334
test


In [2]:
def synchonizerMinute(timeFrame):
    # Alles über h1 funktioniert nicht
    now = datetime.datetime.now()
    print(((((timeFrame-(now.minute % timeFrame))-1) * 60) + (60-now.second))/60)
    time.sleep(((((timeFrame-(now.minute % timeFrame))-1) * 60) + (60-now.second)))

def startScheduler(timeFrame, weekend_trading=False):
    if (not weekend_trading) and datetime.datetime.today().weekday() >=  5:
        now = datetime.datetime.now()
        if now.hours == 23:
            if now.minute <= 50:
                time.sleep((now.minute - 1) * 60)
            else:
                print("System startet gleich")
                ## System wird gleich starten
        else: 
            time.sleep((24-now.hours-1) * 60*60)
    elif (datetime.datetime.today().weekday() < 5) or weekend_trading:
        print("set")
        scheduler.enter(60*timeFrame, 1, print_event, ('first',))
    


In [3]:
import sched
import time

scheduler = sched.scheduler(time.time, time.sleep)

def print_event():
    print ('EVENT:', datetime.datetime.now())
    startScheduler(5)

timeFrame = 5
print (datetime.datetime.today().weekday())
now = datetime.datetime.now()
print(((((5-(now.minute % 5))-1) * 60) + (60-now.second))/60)
print("fertig")
synchonizerMinute(timeFrame)
scheduler.enter(60*timeFrame, 1, print_event, ())


scheduler.run()
print("test")

2
3.9833333333333334
fertig
3.9833333333333334
EVENT: 2021-11-17 00:35:00.458364
set
